In [1]:
url = "http://esl.fis.edu/grammar/q7m/1.htm"

urltmp ="http://esl.fis.edu/grammar/q7m/{}.htm"
print(url)

urls = [  urltmp.format(i) for i in range(1,4)]
urls

http://esl.fis.edu/grammar/q7m/1.htm


['http://esl.fis.edu/grammar/q7m/1.htm',
 'http://esl.fis.edu/grammar/q7m/2.htm',
 'http://esl.fis.edu/grammar/q7m/3.htm']

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep
from collections import defaultdict
import pandas as pd

import pickle
def save(fname, obj):
    with open(fname , 'wb') as f:
        f.write(pickle.dumps(obj))

def load(fname):
    with open(fname , 'rb') as f:
        return pickle.loads(f.read())


In [3]:
def open_url_and_get_soup(url, wait=0, return_driver=False, 
                          load_profiles=False, 
                          user_data_dir=r'/home/bala/.config/google-chrome',
                          profilename='Profile 1'):
    options = webdriver.chrome.options.Options()
    options.add_argument(f"--user-data-dir={user_data_dir}")
    options.add_argument(f'--profile-directory={profilename}')
    
    if load_profiles:
        driver = webdriver.Chrome(options=options)
    else:
        driver = webdriver.Chrome()
    driver.get(url)
    if wait: sleep(wait)
    data = driver.page_source
    soup = BeautifulSoup(data)
    if return_driver:
        return soup, data, driver
    driver.close()
    return soup, data

In [4]:
soup, _, driver = open_url_and_get_soup(url, return_driver=True)

In [5]:
btn = driver.find_element_by_id("checkAnsBut")
btn

<selenium.webdriver.remote.webelement.WebElement (session="5ac9b6331f023701bd6ce48a53dd7e8b", element="f0bc6d10-bf22-4c8c-b280-6f42b9cef65b")>

In [6]:
if btn.is_enabled(): btn.click()

In [7]:
#click the clise button
try:
    driver.find_element_by_xpath('//*[@id="light"]/div[2]/a').click()
except: print("no dialog to close")

In [8]:
soup = BeautifulSoup(driver.page_source)

In [9]:
questions = []

for table in soup.find('form').find_all("table"):
    if table:
        rows = table.find_all("tr")
        q = rows[0].text.strip()
        mc = [ r.text.strip()  for r in rows[1:-1] ]
        ans_tag = table.find("td", {  'style': "background-color: rgb(152, 251, 152);"  })
        if ans_tag:
            ans = ans_tag.text.strip()
        item = (q, *mc, ans )
        questions.append( item )
        print(item)
driver.close()

('Q1. I  .....  tennis every Sunday morning.', 'playing', 'play', 'am playing', 'am play', 'play')
("Q2. Don't make so much noise. Noriko  .....  to study for her ESL test!", 'try', 'tries', 'tried', 'is trying', 'is trying')
('Q3. Jun-Sik  .....  his teeth before breakfast every morning.', 'will cleaned', 'is cleaning', 'cleans', 'clean', 'cleans')
("Q4. Sorry, she can't come to the phone. She  .....  a bath!", 'is having', 'having', 'have', 'has', 'is having')
('Q5. .....  many times every winter in Frankfurt.', 'It snows', 'It snowed', 'It is snowing', 'It is snow', 'It snows')
('Q6. How many students in your class  .....  from Korea?', 'comes', 'come', 'came', 'are coming', 'come')
('Q7. Weather report: "It\'s seven o\'clock in Frankfurt and  .....  ."', 'there is snow', 'it`s snowing', 'it snows', 'it snowed', 'it`s snowing')
('Q8. Babies  .....  when they are hungry.', 'cry', 'cries', 'cried', 'are crying', 'cry')
('Q9. Jane: "What  .....  in the evenings?"Mary: "Usually I watch 

In [10]:
df = pd.DataFrame(questions, columns=['ques','option1', 'option2','option3','option4', 'ans'])

In [11]:
df.head()

,ques,option1,option2,option3,option4,ans
0,Q1. I ..... tennis every Sunday morning.,playing,play,am playing,am play,play
1,Q2. Don't make so much noise. Noriko ..... t...,try,tries,tried,is trying,is trying
2,Q3. Jun-Sik ..... his teeth before breakfast...,will cleaned,is cleaning,cleans,clean,cleans
3,"Q4. Sorry, she can't come to the phone. She ....",is having,having,have,has,is having
4,Q5. ..... many times every winter in Frankfurt.,It snows,It snowed,It is snowing,It is snow,It snows


In [12]:
df.to_csv("grammer_csv.csv", index=False)

## Loop

In [13]:
questions = []

for url in urls:
    soup, _, driver = open_url_and_get_soup(url, return_driver=True)
    btn = driver.find_element_by_id("checkAnsBut")
    if btn.is_enabled(): btn.click()

    try:
        driver.find_element_by_xpath('//*[@id="light"]/div[2]/a').click()
    except: print("no dialog to close")

    soup = BeautifulSoup(driver.page_source)

    for table in soup.find('form').find_all("table"):
        if table:
            rows = table.find_all("tr")
            q = rows[0].text.strip()
            mc = [ r.text.strip()  for r in rows[1:-1] ]
            ans_tag = table.find("td", {  'style': "background-color: rgb(152, 251, 152);"  })
            if ans_tag:
                ans = ans_tag.text.strip()
            item = (q, *mc, ans )
            questions.append( item )
            print(item)

('Q1. I  .....  tennis every Sunday morning.', 'playing', 'play', 'am playing', 'am play', 'play')
("Q2. Don't make so much noise. Noriko  .....  to study for her ESL test!", 'try', 'tries', 'tried', 'is trying', 'is trying')
('Q3. Jun-Sik  .....  his teeth before breakfast every morning.', 'will cleaned', 'is cleaning', 'cleans', 'clean', 'cleans')
("Q4. Sorry, she can't come to the phone. She  .....  a bath!", 'is having', 'having', 'have', 'has', 'is having')
('Q5. .....  many times every winter in Frankfurt.', 'It snows', 'It snowed', 'It is snowing', 'It is snow', 'It snows')
('Q6. How many students in your class  .....  from Korea?', 'comes', 'come', 'came', 'are coming', 'come')
('Q7. Weather report: "It\'s seven o\'clock in Frankfurt and  .....  ."', 'there is snow', 'it`s snowing', 'it snows', 'it snowed', 'it`s snowing')
('Q8. Babies  .....  when they are hungry.', 'cry', 'cries', 'cried', 'are crying', 'cry')
('Q9. Jane: "What  .....  in the evenings?"Mary: "Usually I watch 

In [14]:
df = pd.DataFrame(questions, columns=['ques','option1', 'option2','option3','option4', 'ans'])

In [15]:
df.to_csv("grammer_60.csv", index=False)

In [16]:
!ls

english-grammer-quiz.ipynb  grammer_60.csv  grammer_csv.csv


In [17]:
import numpy as np
def func(x):
    ops = 'option1 	option2 	option3 	option4'.strip().split()
    options = [ x[op] for op in ops ]
    #print(options.index(x.ans))
    return options.index(x.ans)

df["answer"] = df.apply(func, axis=1)

In [18]:
df.head()

,ques,option1,option2,option3,option4,ans,answer
0,Q1. I ..... tennis every Sunday morning.,playing,play,am playing,am play,play,1
1,Q2. Don't make so much noise. Noriko ..... t...,try,tries,tried,is trying,is trying,3
2,Q3. Jun-Sik ..... his teeth before breakfast...,will cleaned,is cleaning,cleans,clean,cleans,2
3,"Q4. Sorry, she can't come to the phone. She ....",is having,having,have,has,is having,0
4,Q5. ..... many times every winter in Frankfurt.,It snows,It snowed,It is snowing,It is snow,It snows,0


In [19]:
df.to_csv("grammer_60.csv", index=False)

In [20]:
df.head()

,ques,option1,option2,option3,option4,ans,answer
0,Q1. I ..... tennis every Sunday morning.,playing,play,am playing,am play,play,1
1,Q2. Don't make so much noise. Noriko ..... t...,try,tries,tried,is trying,is trying,3
2,Q3. Jun-Sik ..... his teeth before breakfast...,will cleaned,is cleaning,cleans,clean,cleans,2
3,"Q4. Sorry, she can't come to the phone. She ....",is having,having,have,has,is having,0
4,Q5. ..... many times every winter in Frankfurt.,It snows,It snowed,It is snowing,It is snow,It snows,0
